In [1]:
!pip install -q \
    torch==1.11.0+cu113 \
    torchvision==0.12.0+cu113 \
    torchaudio==0.11.0 \
    gcc7 \
    opendatasets \
    pycocotools \
    split-folders \
    --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q --upgrade scipy

In [2]:
import torch
#model_inference = torch.hub.load('yolov5', 'custom', path='../weights/exp7/last.pt', source='local')

In [3]:
import os
from flask import Flask, request, jsonify, flash
from src.model import load_model, predict

from werkzeug.utils import secure_filename

app = Flask(__name__)

load_model()

UPLOAD_FOLDER = "/tmp"
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

YOLOv5 🚀 2023-5-9 Python-3.10.8 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 14972MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21091353 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
@app.route('/predict', methods=['POST'])
def predict_duration():
    if 'file' not in request.files:
        return jsonify({}), 400
    
    file = request.files['file']
    filename = os.path.join("/tmp", file.filename)
    file.save(filename)
    predictions = predict(filename)
    # Après la prédiction, on supprime le fichier
    os.remove(filename)

    # Pour chaque prédiction, on a :
    # (catégorie, [x, y, w, h, probabilité])
    return jsonify(predictions if predictions else []), 200

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
